<a href="https://colab.research.google.com/github/ra4ola/APA-Assigment2/blob/main/Parte%201/Parte1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
from torch.nn.modules.flatten import Flatten
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import os
from google.colab import drive
import torch.nn.functional as F
from sklearn.manifold import TSNE


# Mount Drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


# Definição de Variáveis

In [3]:
PATH = '/content/drive/MyDrive/APA/Assigment2/Parte3'

In [4]:
TRAIN = True

In [5]:
image_size = 28
lr = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)

# Modelos


In [6]:
# @title AE

class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()
        self.encoder = nn.Sequential(

            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(8),
            nn.ELU(),

            nn.Conv2d(in_channels=8, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ELU(),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ELU(),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ELU(),

            nn.Conv2d(in_channels=128, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ELU()
        )
        self.decoder = nn.Sequential(

         nn.ConvTranspose2d(in_channels=16, out_channels=128, kernel_size=3, stride=1, padding=1),
         nn.BatchNorm2d(128),
         nn.ELU(),

         nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1),
         nn.BatchNorm2d(64),
         nn.ELU(),

         nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1),
         nn.BatchNorm2d(32),
         nn.ELU(),

         nn.ConvTranspose2d(in_channels=32, out_channels=8, kernel_size=3, stride=1, padding=1),
         nn.BatchNorm2d(8),
         nn.ELU(),

         nn.Upsample(scale_factor=2, mode='nearest'),  # Upsample
         nn.ConvTranspose2d(in_channels=8, out_channels=1, kernel_size=3, stride=1, padding=1),
         nn.BatchNorm2d(1),
         nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [7]:
# @title VAE

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # Encoder
        self.encoder = AE()

        # Latent space parameters
        self.fc_mean = nn.Linear(3136, 16)
        self.fc_logvar = nn.Linear(3136, 16)

        # Decoder
        self.decoder_fc = nn.Linear(16,3136)
        self.decoder = AE()

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        epsilon = torch.randn_like(std)
        return mean + epsilon * std

    def forward(self, x):
        # Encoder
        x = self.encoder.encoder(x)
        x = x.view(x.size(0), -1)

        # Latent space parameters
        mean, logvar = self.fc_mean(x), self.fc_logvar(x)

        # Reparameterization trick
        z = self.reparameterize(mean, logvar)

        z = self.decoder_fc(z)


        z = z.view(z.size(0),16, 14, 14)

        x_recon = self.decoder.decoder(z)

        return x_recon, mean, logvar



In [8]:
# @title DAE

class DAE(nn.Module):
    def __init__(self):
      super(DAE, self).__init__()
      self.encoder = AE()

    def forward(self, x):
      x = self.encoder(x)
      return x

# Definição de Funções

### Funções de Construção das String para os Caminhos

In [9]:
def ModelDirStr(modelName,path = PATH):
  return f"{path}/{modelName}"
def pathModelStr(modelName,epoch, path = PATH):
  return f"{ModelDirStr(modelName,path)}/{modelName}_epoch={epoch}"

### Função para adicionar Noise

In [10]:
# @title Noise Fuction DAE

def add_noise(data, noise_factor=0.5):
    noisy_data = data + noise_factor * torch.randn_like(data)
    return torch.clamp(noisy_data, 0., 1.)




### Função de Treino


In [11]:
# @title Train Fuction

def train(model, criterion, optimizer, train_loader, epochs, modelName, scheduler=None, path=PATH):
    os.makedirs(ModelDirStr(modelName=modelName, path=path), exist_ok=True)

    for epoch in range(epochs):
        model.train()  # Set the model to training mode

        epoch_loss = 0

        for inputs, _ in train_loader:
            inputs = inputs.to(device)
            optimizer.zero_grad()

            if isinstance(model, VAE):
                outputs, mean, log_var = model(inputs)
                reconstruction_loss = criterion(outputs, inputs)
                kld_loss = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
                loss = reconstruction_loss + kld_loss
            else:
                outputs = model(inputs)
                loss = criterion(outputs, inputs)

            loss.backward()
            optimizer.step()

            epoch_loss = epoch_loss =+ loss.item()

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

        if scheduler is not None:
            scheduler.step()

        torch.save(model, pathModelStr(modelName=modelName, path=path, epoch=epoch))


### Função de Implementação das Perdas

In [12]:
# @title Evalute Training

def evaluate_training(training_loader, loss_fn, num_epochs, modelName, path=PATH):
    all_losses = []
    all_kl_loss = []

    # Load the model outside the epoch loop


    for epoch in range(num_epochs):
        running_loss = 0.0
        running_kld = 0.0
        model_path = pathModelStr(modelName=modelName, path=path, epoch=epoch)
        model = torch.load(model_path).to(device)

        model.eval()

        with torch.no_grad():
            for batch in training_loader:
                inputs, targets = batch
                inputs = inputs.to(device)

                if isinstance(model, VAE):
                    outputs, mean, log_var = model(inputs)
                    reconstruction_loss = loss_fn(outputs, inputs)
                    kld_loss = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
                    loss = reconstruction_loss + kld_loss
                else:
                    outputs = model(inputs)
                    loss = loss_fn(outputs, inputs)

                running_loss += loss.item()
                running_kld += kld_loss.item() if isinstance(model, VAE) else 0

        all_losses.append(running_loss)
        all_kl_loss.append(running_kld)

        print(f'Epoch {epoch}: Epoch Loss: {running_loss/len(train_loader):.4f}, KL Divergence: {running_kld:.4f}')

    return all_losses, all_kl_loss



### Função Display Images

In [13]:
# @title Plot Losses

def plot_loss_accuracy(losses, Tittle = 'Traning Loss', curve_Label = 'Loss',X_axis_label = 'Epochs', Y_axis_Label = 'Loss'):
    epochs = range(1, len(losses) + 1)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, losses, 'b', label=curve_Label)
    plt.title(Tittle)
    plt.xlabel(X_axis_label)
    plt.ylabel(Y_axis_Label)
    plt.legend()

    plt.tight_layout()
    plt.show()

In [14]:
# @title Image Reconstruction

def reconstruction_images_plot(model,test_loader, Title = 'Reconstruction'):
  with torch.no_grad():
    for data in test_loader:
      inputs, _ = data
      inputs = inputs.to(device)

      if isinstance(model,VAE):
        outputs,_,_ = model(inputs)
      else:
        outputs = model(inputs)

      print(outputs.shape)
      n = 10
      plt.figure(figsize=(20, 4))
      plt.title = Title
      for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(inputs[i].cpu().view(28, 28).numpy(), cmap='gray')
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(outputs[i].cpu().view(28, 28).numpy(), cmap='gray')
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
      plt.show()
      break

# Importação do DataSet

In [15]:
# @title DataSet Import

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 84880956.87it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 105061312.94it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 24339046.54it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 19050528.77it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



# Implementação dos Modelos

## Inicialização do Modelo

In [16]:
# @title AE Initialization

epochs = 50
ae = AE().to(device)

criterion = nn.MSELoss()

optimizerAE = optim.Adam(ae.parameters(), lr = lr)

## AE Treino

In [17]:
# @title Treino AE
train(
      model = ae,
      criterion = criterion,
      optimizer = optimizerAE,
      train_loader = train_loader,
      epochs = epochs,
      modelName = "AE",
      scheduler=None,
      path = PATH
    )

Epoch [1/50], Loss: 1.3421
Epoch [2/50], Loss: 1.1832
Epoch [3/50], Loss: 1.0516
Epoch [4/50], Loss: 1.0070
Epoch [5/50], Loss: 0.9655
Epoch [6/50], Loss: 0.9279
Epoch [7/50], Loss: 0.8941
Epoch [8/50], Loss: 0.8857
Epoch [9/50], Loss: 0.8754
Epoch [10/50], Loss: 0.8643
Epoch [11/50], Loss: 0.8712
Epoch [12/50], Loss: 0.8558
Epoch [13/50], Loss: 0.8497
Epoch [14/50], Loss: 0.8438
Epoch [15/50], Loss: 0.8478
Epoch [16/50], Loss: 0.8482
Epoch [17/50], Loss: 0.8393
Epoch [18/50], Loss: 0.8346
Epoch [19/50], Loss: 0.8416
Epoch [20/50], Loss: 0.8390
Epoch [21/50], Loss: 0.8288
Epoch [22/50], Loss: 0.8350
Epoch [23/50], Loss: 0.8305
Epoch [24/50], Loss: 0.8330
Epoch [25/50], Loss: 0.8297
Epoch [26/50], Loss: 0.8330
Epoch [27/50], Loss: 0.8323
Epoch [28/50], Loss: 0.8330
Epoch [29/50], Loss: 0.8304
Epoch [30/50], Loss: 0.8360
Epoch [31/50], Loss: 0.8479
Epoch [32/50], Loss: 0.8309
Epoch [33/50], Loss: 0.8331
Epoch [34/50], Loss: 0.8311
Epoch [35/50], Loss: 0.8438
Epoch [36/50], Loss: 0.8340
E

In [21]:
# @title AE Losses Calculation

loss,_ =evaluate_training(
    training_loader = train_loader,
    loss_fn = criterion,
    num_epochs = epochs,
    modelName = "AE",
    path = PATH
    )


Epoch 0: Epoch Loss: 1.3712, KL Divergence: 0.0000
Epoch 1: Epoch Loss: 1.1800, KL Divergence: 0.0000
Epoch 2: Epoch Loss: 1.0683, KL Divergence: 0.0000
Epoch 3: Epoch Loss: 1.0010, KL Divergence: 0.0000
Epoch 4: Epoch Loss: 0.9544, KL Divergence: 0.0000
Epoch 5: Epoch Loss: 0.9226, KL Divergence: 0.0000
Epoch 6: Epoch Loss: 0.9027, KL Divergence: 0.0000
Epoch 7: Epoch Loss: 0.8870, KL Divergence: 0.0000
Epoch 8: Epoch Loss: 0.8747, KL Divergence: 0.0000
Epoch 9: Epoch Loss: 0.8666, KL Divergence: 0.0000
Epoch 10: Epoch Loss: 0.8590, KL Divergence: 0.0000
Epoch 11: Epoch Loss: 0.8543, KL Divergence: 0.0000
Epoch 12: Epoch Loss: 0.8505, KL Divergence: 0.0000
Epoch 13: Epoch Loss: 0.8461, KL Divergence: 0.0000
Epoch 14: Epoch Loss: 0.8435, KL Divergence: 0.0000
Epoch 15: Epoch Loss: 0.8414, KL Divergence: 0.0000
Epoch 16: Epoch Loss: 0.8399, KL Divergence: 0.0000
Epoch 17: Epoch Loss: 0.8385, KL Divergence: 0.0000
Epoch 18: Epoch Loss: 0.8375, KL Divergence: 0.0000
Epoch 19: Epoch Loss: 

In [ ]:
# @title AE Plot Losses
plot_loss_accuracy(losses = loss, Tittle = 'AE Loss')

## DAE Treino

In [19]:
# @title DAE Initialization

dae = DAE().to(device)

optimizerDAE = optim.Adam(dae.parameters(), lr = lr)

In [20]:
# @title Treino DAE

train(
    model = dae,
    criterion = criterion,
    optimizer = optimizerDAE,
    train_loader = train_loader,
    epochs = epochs,
    modelName = "DAE",
    scheduler=None,
    path = PATH
  )

Epoch [1/50], Loss: 1.3621
Epoch [2/50], Loss: 1.1901
Epoch [3/50], Loss: 1.0533
Epoch [4/50], Loss: 1.0033
Epoch [5/50], Loss: 0.9532
Epoch [6/50], Loss: 0.9138
Epoch [7/50], Loss: 0.9141
Epoch [8/50], Loss: 0.8785
Epoch [9/50], Loss: 0.8791
Epoch [10/50], Loss: 0.8758
Epoch [12/50], Loss: 0.8439
Epoch [13/50], Loss: 0.8605
Epoch [14/50], Loss: 0.8396
Epoch [15/50], Loss: 0.8500
Epoch [16/50], Loss: 0.8318
Epoch [17/50], Loss: 0.8404
Epoch [18/50], Loss: 0.8456
Epoch [19/50], Loss: 0.8373
Epoch [20/50], Loss: 0.8391
Epoch [21/50], Loss: 0.8356
Epoch [22/50], Loss: 0.8478
Epoch [23/50], Loss: 0.8393
Epoch [24/50], Loss: 0.8270
Epoch [25/50], Loss: 0.8318
Epoch [26/50], Loss: 0.8326
Epoch [27/50], Loss: 0.8317
Epoch [28/50], Loss: 0.8280
Epoch [29/50], Loss: 0.8336
Epoch [30/50], Loss: 0.8357
Epoch [31/50], Loss: 0.8385
Epoch [32/50], Loss: 0.8370
Epoch [33/50], Loss: 0.8348
Epoch [34/50], Loss: 0.8358
Epoch [35/50], Loss: 0.8295
Epoch [36/50], Loss: 0.8345
Epoch [37/50], Loss: 0.8246
E

In [ ]:
# @title DAE Losses Calculation

loss,_ =evaluate_training(
    training_loader = train_loader,
    loss_fn = criterion,
    num_epochs = epochs,
    modelName = "DAE",
    path = PATH
  )

In [ ]:
# @title DAE Plot Losses
plot_loss_accuracy(losses = loss)

## VAE Treino

In [ ]:
# @title Initialization VAE
vae = VAE().to(device)

optimizerVAE = optim.Adam(vae.parameters(), lr = lr)

In [ ]:
# @title VAE Treino
train(
    model = vae,
    criterion = criterion,
    optimizer = optimizerVAE,
    train_loader = train_loader,
    epochs = epochs,
    modelName = "VAE",
    scheduler=None,
    path = PATH
  )

In [ ]:
# @title VAE Calculate Losses

loss,kld_loss =evaluate_training(
    training_loader = train_loader,
    loss_fn = criterion,
    num_epochs = epochs,
    modelName = "VAE",
    path = PATH
  )

In [ ]:
# @title VAE Plot Losses

plot_loss_accuracy(losses= loss)

In [ ]:
# @title Reconstrução das Imagens
reconstruction_images_plot(vae,test_loader)

# Latent Space

In [ ]:
# @title Latent Space Calculation  and Plot Fuction
def extract_latent_space_and_labels_with_tsne(model, training_loader, device):
    latent_space = []
    labels = []

    with torch.no_grad():
        for batch_idx, data in enumerate(training_loader):
            model.eval()
            mu = model.encoder(data.to(device))
            mu = mu.view(mu.size(0), -1)[:, :2]

            latent_space.append(mu)
            labels.append(data)

    latent_space = torch.cat(latent_space, dim=0).cpu().numpy()
    labels = torch.cat(labels, dim=0).cpu().numpy()

    # Apply t-SNE
    tsne = TSNE(n_components=2, random_state=42)
    latent_tsne = tsne.fit_transform(latent_space)

    # Plot the t-SNE visualization
    plt.scatter(latent_tsne[:, 0], latent_tsne[:, 1], c=labels, cmap='viridis')
    plt.colorbar()
    plt.title("t-SNE Visualization of Latent Space")
    plt.show()

    return latent_space, labels, latent_tsne

In [ ]:
latent_spaceae, labelsae, latent_tsneae = extract_latent_space_and_labels_with_tsne(ae, test_dataset, device)

In [ ]:
latent_spacedae, labelsdae, latent_tsnedae = extract_latent_space_and_labels_with_tsne(dae, test_dataset, device)

In [ ]:
latent_spacedvae, labelsvae, latent_tsnevae = extract_latent_space_and_labels_with_tsne(vae, test_dataset, device)